## Задание 
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.
3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from fake_headers import Headers

# для взаимодействия с MongoDB на удалённом сервере
import pymongo
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import configparser

headers = Headers(headers=True).generate()

In [2]:
def add_to_server(vacancy_title: str, vacancy_url: str, max_salary: str, min_salary: str, town: str):  
    """Подключение к удалённому серверу MongoDB и запись собранных вакансий"""
    
    config = configparser.ConfigParser()
    config.read('config.ini')

    MONGO_HOST = config['MongoDB_server']['MONGO_HOST']
    MONGO_USER = config['MongoDB_server']['MONGO_USER']
    MONGO_PASS = config['MongoDB_server']['MONGO_PASS']

    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('127.0.0.1', 27017)
        )

    server.start()

    client = MongoClient('127.0.0.1', server.local_bind_port)

        # Выбор базы данных для работы. Если базы данных не существовало, то она  будет создана.
    db = client.lesson_3    

        
    vacancy = db.vacancy
    vacancy.insert_one(
        {"vacancy_title": vacancy_title,
        "vacancy_url":  vacancy_url,
        "max_salary": max_salary,
        "min_salary": min_salary, 
        "town":town
        }
        )    
        
    server.stop()

In [3]:
def find_on_server(salary_min: str = ''):  
    """Подключение к удалённому серверу MongoDB 
    и поиск вакансий с заработной платой больше введённой суммы"""

    config = configparser.ConfigParser()
    config.read('config.ini')

    MONGO_HOST = config['MongoDB_server']['MONGO_HOST']
    MONGO_USER = config['MongoDB_server']['MONGO_USER']
    MONGO_PASS = config['MongoDB_server']['MONGO_PASS']

    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('127.0.0.1', 27017)
        )

    server.start()

    client = MongoClient('127.0.0.1', server.local_bind_port)

    # Выбор базы данных для работы. Если базы данных не существовало, то она  будет создана.
    db = client.lesson_3    
    vacancy = db.vacancy

    if salary_min == 0:
        for x in vacancy.find(): 
            print(x)
    else:
        for x in vacancy.find({"min_salary": {'$gte': salary_min}}):
            print(x)
    server.stop()

In [4]:
def scrap_hh(search_keywords: str, new_vacancy: bool = False):
    """
    Сбор информации о вакансиях на вводимую должность с сайта HH 
    и запись собранных вакансий в БД
    При new_vacancy = True в БД добавляются только новые вакансии с сайта.
    """
    if new_vacancy: 
        url_hh = f"https://hh.ru/search/vacancy?clusters=true&enable_snippets=true&text={search_keywords}&showClusters=true"
    else:
        url_hh = f"https://hh.ru/search/vacancy?clusters=true&enable_snippets=true&text={search_keywords}&showClusters=true&search_period=1"

    response = requests.get(url_hh, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    jobs_block = soup.find('div',{'class':'vacancy-serp'})
    jobs_list = jobs_block.findChildren(recursive=False)

    for job in jobs_list:
        holder=job.find('span',{'class':'g-user-content'})
        if holder!=None:
            main_info = holder.findChild()
            vacancy_title = main_info.getText()
            vacancy_url = main_info['href']
            salary = job.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            town = job.find('span',{'data-qa':'vacancy-serp__vacancy-address'}).text

            # Получаем максимальную и минимальную зп
            if salary:
                salary = salary.getText()
                # удаляем пробелы, буквы и точки
                salary = ''.join(i for i in salary if not i.isalpha())
                salary = salary.replace('\u202f', '')
                salary = salary.replace(' ', '')
                salary = salary.replace('.', '')
                salary = salary.split('–')
                min_salary = salary[0]
                if len(salary) > 1:
                    max_salary = salary[1]
                else:
                    max_salary = None
            else:
                max_salary = None
                min_salary = None

        add_to_server(vacancy_title, vacancy_url, max_salary, min_salary, town)

In [5]:
def scrap_superjob(search_keywords: str, new_vacancy: bool = False):
    """
    Сбор информации о вакансиях на вводимую должность с сайта Superjob 
    и запись собранных вакансий в БД.
    При new_vacancy = True в БД добавляются только новые вакансии с сайта.

    """
    if new_vacancy:
        superjob_url = f'https://russia.superjob.ru/vacancy/search/?keywords={search_keywords}&period=1'
    else:
        superjob_url = f'https://russia.superjob.ru/vacancy/search/?keywords={search_keywords}'

    response_superjob = requests.get(superjob_url, headers=headers)
    soup = BeautifulSoup(response_superjob.text, 'lxml')
    holder = soup.find(class_='_1Ttd8 _2CsQi')
    items = holder.find_all('div', {'class': '_31XDP iJCa5 f-test-vacancy-item _1fma_ _2nteL'})

    for item in items:
        vacancy_title = item.find(class_='icMQ_').text
        vacancy_url = 'https://russia.superjob.ru' + item.find(class_='icMQ_')['href']
        town = [i.text for i in item.select('div.f-test-search-result-item > div >'
                                            'div:nth-child(1) > div:nth-child(1) >'
                                            'div:nth-child(1) > div:nth-child(3) >' 
                                            'div:nth-child(1) > div:nth-child(2) >'
                                            'div:nth-child(2) > span:nth-child(1)>'
                                            'span:nth-child(3)')]

        # Получаем максимальную и минимальную зп
        salary = item.find(class_='_1h3Zg _2Wp8I _2rfUm _2hCDz _2ZsgW').text
        salary = ''.join(i for i in salary if not i.isalpha())
        salary = salary.replace('\xa0', '')
        salary = salary.replace(' ', '')
        salary = salary.replace('.', '')
        if salary:
            salary = salary.split('—')
            min_salary = salary[0]
            if len(salary) > 1:
                max_salary = salary[1]
            else: 
                max_salary = None
        else:
            max_salary = None
            min_salary = None


        add_to_server(vacancy_title, vacancy_url, max_salary, min_salary, town)

In [6]:
# Соберём и запишем в базу данных вакансии с сайтов Superjob и HH.
search = 'python'

scrap_hh(search, new_vacancy = True)
scrap_superjob(search, new_vacancy = True)

In [7]:
# Произведём поиск и выведем  вакансии с заработной платой больше 50000.
find_on_server("50000")

{'_id': ObjectId('61114219284fa214614e8fb3'), 'vacancy_title': 'Senior Python developer', 'vacancy_url': 'https://kaluga.hh.ru/vacancy/46911211?from=vacancy_search_list&query=python', 'max_salary': None, 'min_salary': '5500', 'town': 'Санкт-Петербург'}
{'_id': ObjectId('6111421a284fa214614e8fb5'), 'vacancy_title': 'Senior Python developer', 'vacancy_url': 'https://kaluga.hh.ru/vacancy/46911211?from=vacancy_search_list&query=python', 'max_salary': None, 'min_salary': '5500', 'town': 'Санкт-Петербург'}
{'_id': ObjectId('6111423e284fa214614e8fe9'), 'vacancy_title': 'Python разработчик (senior) (backend)', 'vacancy_url': 'https://kaluga.hh.ru/vacancy/43941903?from=vacancy_search_list&query=python', 'max_salary': None, 'min_salary': '500000', 'town': 'Москва, Площадь Гагарина'}
{'_id': ObjectId('61114244284fa214614e8fef'), 'vacancy_title': 'Программист Android / Python (удаленная работа)', 'vacancy_url': 'https://russia.superjob.ru/vakansii/programmist-android-38284380.html', 'max_salary': 